In [10]:
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
import pandas as pd
from package.mdtool.mdtool import load_traj, traj_slice
from package.utils.utils import filter_digits

In [48]:
def dict_map(d: dict, x, f=float):
    return np.array(list(map(d.__getitem__, x))).astype(f)

# helper functions
amino_abr = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS", "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"]
amino_code = ["A", "R", "N", "D", "C", "E", "Q", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]


abr_to_code=dict(zip(amino_abr, amino_code))
code_to_abr=dict(zip(amino_code, amino_abr))
code_to_numerical = dict(zip(amino_code, range(20)))

to_code = lambda x : dict_map(abr_to_code, x, str)
to_numerical = lambda x : dict_map(code_to_numerical, x, float)
to_seq = lambda x : dict_map(code_to_abr, x, str)

def convolve(x, dx):
    return np.array([(x[i:i+len(dx)].flatten() == dx.flatten()).sum()
                    for i in range(len(x) - len(dx) + 1)])

In [33]:
pdb = md.load("frame_0.pdb")
ntail_seq = [filter_digits(str(i)) for i in list(traj_slice(pdb, "chainid 1").top.residues)]
xd_seq = [filter_digits(str(i)) for i in traj_slice(pdb, "chainid 0").top.residues]
ntail_code = to_code(ntail_seq)
xd_code = to_code(xd_seq)

 # independent structure of entire sendai nucleoprotein from protein data base

In [34]:
fasta = "MAGLLSTFDTFSSRRSESINKSGGGAVIPGQRSTVSVFVLGPSVTDDADKLSIATTFLAHSLDTDKQHSQRGGFLVSLLAMAYSSPELYLTTNGVNADVKYVIYNIEKDPKRTKTDGFIVKTRDMEYERTTEWLFGPMVNKSPLFQGQRDAADPDTLLQIYGYPACLGAIIVQVWIVLVKAITSSAGLRKGFFNRLEAFRQDGTVKGALVFTGETVEGIGSVMRSQQSLVSLMVETLVTMNTARSDLTTLEKNIQIVGNYIRDAGLASFMNTIKYGVETKMAALTLSNLRPDINKLRSLIDTYLSKGPRAPFICILKDPVHGEFAPGNYPALWSYAMGVAVVQNKAMQQYVTGRTYLDMEMFLLGQAVAKDAESKISSALEDELGVTDTAKERLRHHLANLSGGDGAYHKPTGGGAIEVALDNADIDLEPEAHTDQDARGWGGDSGDRWARSMGSGHFITLHGAERLEEETNDEDVSDIERRIARRLAERRQEDATTHEDEGRNNGVDHDEEDDAAAAAGMGGI"
exp_code = [str(i) for i in fasta]

load_traj("./", traj_keyword="traj_trunc", pdb_keyword="frame")

 # Blackledge paper sequence - says that molecular recognition starts as index 8

In [35]:
paper_code = ["E", "E", "E", "T", "N", "D", "E", "D", "V", "S",
             "D", "I", "E","R", "R", "I", "A", "M", "R", "L", 
             "A", "E", "R", "R", "Q", "E", "D", "S", "A", "T", "H",
             "G", "D"]
paper_seq = to_seq(paper_code)

paper_nt_code, paper_nt_seq = (i[8:28] for i in (paper_code, paper_seq))

In [36]:
paper_num, exp_num = (to_numerical(i) for i in (paper_nt_code, exp_code))

# Sequence in slack matches blackledge (20 residues)

In [37]:
test_seq = to_numerical(list(map(str, "VSDIERRIAMRLAERRQEDS")))

#(exp_num[475:496]==sim_num).sum()
print(f"Match : {(test_seq==paper_num).sum()} / 20")

Match : 20 / 20


 # Matches between blackledge and additional database sequence for entire nucleoprotein

In [38]:
matches_pdb_nmr = convolve(exp_num, paper_num)
print(f"max seq match : {matches_pdb_nmr.max()}, start index : {matches_pdb_nmr.argmax()}")

max seq match : 18, start index : 475


  # Matches between blackledge Ntail  and our simulation Ntail

In [39]:
paper_num_full, sim_num = (to_numerical(i) for i in (paper_code, ntail_code))
matches_sendai_measles = convolve(paper_num_full, sim_num)
print(f"max seq match : {matches_sendai_measles.max()}, start index : {matches_sendai_measles.argmax()}")

max seq match : 5, start index : 8


 # Paper states that molecular recognition residue starts at 8, so this checks out

In [40]:
pd.DataFrame(data=np.stack([to_seq(i) for i in [ntail_code[:-1], paper_nt_code, exp_code[475:495]]], 1),
             columns="Measles,Sendai (Blackledge),Sendai (PDB)".split(","),
            )

,Measles,Sendai (Blackledge),Sendai (PDB)
0,GLY,VAL,VAL
1,SER,SER,SER
2,GLN,ASP,ASP
3,ASP,ILE,ILE
4,SER,GLU,GLU
5,ARG,ARG,ARG
6,ARG,ARG,ARG
7,SER,ILE,ILE
8,ALA,ALA,ALA
9,ASP,MET,ARG


# XD / PX Sequence : the PDB sequence was sumbitted by Backledge et al

In [41]:
xd_pdb_code = list(map(str, "KPTMHSLRLVIESSPLSRAEKAAYVKSLSKCKTDQEVKAVMELVEEDIESLTN"))
xd_pdb_num = to_numerical(xd_pdb_code)
xd_sim_num = to_numerical(xd_code)

In [42]:
matches_pdb_sim = convolve(xd_pdb_num, xd_sim_num)
print(f"max seq match : {matches_pdb_sim.max()}, start index : {matches_pdb_sim.argmax()}")

max seq match : 9, start index : 0


In [43]:
pd.DataFrame(data=np.stack([xd_seq, to_seq(xd_pdb_code[:-4])], 1),
            columns="Measles,Sendai".split(","))

,Measles,Sendai
0,GLY,LYS
1,ALA,PRO
2,SER,THR
3,ARG,MET
4,SER,HIS
5,VAL,SER
6,ILE,LEU
7,ARG,ARG
8,SER,LEU
9,ILE,VAL
